In [ ]:
#FITE7410 Financial Fraud Analytics First Semester, 2025-2026 
# Assignnment 1 Exploratory Data Analysis (EDA)
# Student: Choi Chi For 3036384024
# Due Date: 19 Oct, 2025 (Sun) 23:59
# Objective of this assignment:
# 1. Perform data cleaning and preparation
# 2. Explore and visualize the data to identify patterns and trends
# 3. Engineer new features based on domain knowledge or insights from EDA
# 4. Prepare a report summarizing the findings from EDA

# Load necessary libraries -----------------------
install.packages("dplyr")
install.packages("tidyr")
install.packages("ggplot2")
install.packages("corrplot")
install.packages("tidyverse")
library(dplyr)
library(tidyr)
library(ggplot2)
library(corrplot)
library(tidyverse)  

# build the necessary functions -----------------------
get_mode <- function(aVector) {
  uniqv <- unique(aVector)
  uniqv[which.max(tabulate(match(aVector, uniqv)))]
}

# Load Data -----------------------
tryCatch({
  data <- read.csv("A1_data.csv")
}, warning = function(w) {
  print(w)
}, error = function(e) {
  print(e)
}, finally = {
print("Data loading attempt finished.")
})

#str(data)

aSummary <- data.frame(summary(data))
aSummary